# Experiment 01

First Tests from the predictions.

In [1]:
import numpy as np
import pandas as pd
import math

import sys
sys.path.insert(1, 'C:\\Users\\Adrion\\Desktop\\TG\\DayTradeSystem\\Part1_Prediction')
import Utils 
from Utils import calculateEMA, calculateBB, addPreviousDays, removeColumns, getPeriod, mape, mae, createDate, getDate, printResult

import matplotlib.pyplot as plt
import matplotlib.dates as mpdates
import mplfinance as mpf
from mplfinance.original_flavor import candlestick_ohlc

import warnings
warnings.filterwarnings('ignore')


def getPeriod2(df, begin, end, resetIndex = False):
    """
    Returns the df in the chosen interval
    
    Object begin: Start date forrmated as 'yyyy.mm.dd'.
    Object   end: End date 'yyyy.mm.dd'.

    returns a dataframe with the historic of the selected period
    """
    
    indexBegin = df[df['date']==begin].index[0]
    indexEnd = df[df['date']==end].tail(1).index[0]
    
    if (resetIndex):
        return df[(df.index >= indexBegin) & (df.index <= indexEnd)].reset_index(drop=True)
    else: 
        return df[(df.index >= indexBegin) & (df.index <= indexEnd)]

In [2]:
dfColumns = ['date','time','open', 'high', 'low','close','tickvol','volume','spread']
newColumns = ['open', 'high', 'low', 'close']

TEST_BEGIN = '2021.04.06'
TEST_END = '2021.06.01'

TRAIN_BEGIN = '2020.03.31'
TRAIN_END = '2021.05.31'

TIMES = 10

In [3]:
df = pd.read_csv('../Data/PETR4_M15.csv', sep = "\t")
df.columns = dfColumns
df = removeColumns(df, ['tickvol', 'volume','spread'])

df2 = pd.read_csv('../Data/VALE3_M15.csv', sep = "\t")
df2.columns = dfColumns
df2 = removeColumns(df2, ['tickvol', 'volume','spread'])

initialDf_PETR4 = getPeriod2(df, TEST_BEGIN, TEST_END)
initialDf_VALE3 = getPeriod2(df2, TEST_BEGIN, TEST_END)

In [4]:
initialDf_PETR4

,date,time,open,high,low,close
6197,2021.04.06,10:00:00,23.39,23.47,23.32,23.32
6198,2021.04.06,10:15:00,23.32,23.38,23.30,23.36
6199,2021.04.06,10:30:00,23.36,23.53,23.33,23.52
6200,2021.04.06,10:45:00,23.52,23.58,23.39,23.48
6201,2021.04.06,11:00:00,23.48,23.49,23.36,23.39
...,...,...,...,...,...,...
7311,2021.06.01,15:45:00,27.14,27.14,27.05,27.06
7312,2021.06.01,16:00:00,27.06,27.09,27.01,27.02
7313,2021.06.01,16:15:00,27.01,27.17,27.01,27.14
7314,2021.06.01,16:30:00,27.15,27.31,27.11,27.27


In [5]:
initialDf_VALE3

,date,time,open,high,low,close
6186,2021.04.06,10:00:00,103.80,104.35,103.52,103.77
6187,2021.04.06,10:15:00,103.77,103.80,102.92,103.23
6188,2021.04.06,10:30:00,103.23,104.25,103.00,104.23
6189,2021.04.06,10:45:00,104.23,104.56,103.70,104.03
6190,2021.04.06,11:00:00,104.03,104.12,103.15,103.34
...,...,...,...,...,...,...
7301,2021.06.01,15:45:00,113.84,113.95,113.61,113.72
7302,2021.06.01,16:00:00,113.72,114.00,113.70,113.99
7303,2021.06.01,16:15:00,113.99,114.00,113.90,113.96
7304,2021.06.01,16:30:00,113.97,114.00,113.88,113.88


In [6]:
pred_PETR4 = pd.read_pickle('../Data/PETR4_Pred.pkl')
pred_PETR4.head(5)

,high,low,high_pred,low_pred,date
559,23.58,23.13,23.751067,23.068200,2021.04.06
560,23.55,23.15,23.688000,22.981167,2021.04.07
561,23.26,22.61,23.699267,22.992433,2021.04.08
562,23.01,22.61,23.417233,22.778233,2021.04.09
563,23.58,23.01,23.529767,22.874833,2021.04.12


In [7]:
pred_VALE3 = pd.read_pickle('../Data/VALE3_Pred.pkl')
pred_VALE3.head(15)

,high,low,high_pred,low_pred,date
559,104.56,102.05,102.215667,98.809267,2021.04.06
560,105.32,102.15,102.421767,98.959767,2021.04.07
561,105.30,104.12,102.250167,98.949867,2021.04.08
562,104.66,102.94,102.272433,98.989200,2021.04.09
563,103.63,102.21,102.240133,98.956067,2021.04.12
564,104.29,102.85,102.240133,98.956067,2021.04.13
565,107.78,103.77,102.240133,98.956067,2021.04.14
566,109.88,107.55,102.240133,98.956067,2021.04.15
567,109.11,107.50,102.240133,98.956067,2021.04.16
568,109.88,107.64,102.240133,98.956067,2021.04.19


### Início do sistema

In [8]:
def getDay(df, day):
    return df[df['date']==day]

def buy(value, availableMoney, opr='normal'):
    lotValue = value*100
    quantity = math.floor(availableMoney/lotValue)
    
    amount = quantity*lotValue
    remainingMoney = availableMoney - amount
    
    return remainingMoney, quantity

def sell(value, amount, opr='normal'):
    if(opr == 'normal'):
        return value*amount*100
    else:
        return value*amount*100

In [9]:
def runStrategy(df, dailyDf, Money, stopLoss, std):
    size = len(df.index)

    Money = Money
    stopLoss = stopLoss

    orders = []

    for day in range(0,size):
        currentDay =  df.iloc[day].date

        lowPred = df.iloc[day].low_pred*(1+std)
        highPred = df.iloc[day].high_pred*(1-std)

        dayDf = getPeriod2(dailyDf, currentDay, currentDay)
        daySize = len(dayDf.index)

        op = 'none'
        quantity = 0
        trade = []

        for i in range(0,daySize):
            currentTime = dayDf.iloc[i]

            if((op == 'buy') and (currentTime.close < opPrice*(1-stopLoss) or currentTime.close >= highPred or i == daySize-1)):
                isStop = False
                if(currentTime.close < opPrice*(1-stopLoss)): isStop = True
                aux = sell(currentTime.close, quantity)
                Money = Money + aux 
                trade = trade + [currentTime.time, 'Sell', currentTime.close, currentTime.close-opPrice,(currentTime.close-opPrice)*100*quantity, isStop, Money]
                orders.append(trade)

                quantity = 0
                op = 'none'

            elif((op == 'sell') and (currentTime.close > opPrice*(1+stopLoss) or currentTime.close <= lowPred or i == daySize-1)):
                isStop = False
                if(currentTime.close > opPrice*(1+stopLoss)): isStop = True

                aux = sell(currentTime.close, quantity)
                Money = Money + (opPrice*100*quantity) + ((opPrice*100*quantity) - aux) 
                trade = trade + [currentTime.time, 'Buy', currentTime.close, opPrice-currentTime.close,(opPrice-currentTime.close)*100*quantity, isStop, Money]
                orders.append(trade)

                quantity = 0
                op = 'none'

            elif(op == 'none' and i < daySize-1 ):
                if(currentTime.close <= lowPred):
                    Money, quantity = buy(currentTime.close, Money)
                    opPrice = currentTime.close
                    op = 'buy'

                    trade = [currentTime.date, quantity, currentTime.time, 'Buy', currentTime.close]

                elif(currentTime.close >=  highPred):
                    Money, quantity = buy(currentTime.close, Money)
                    opPrice = currentTime.close
                    op = 'sell'

                    trade = [currentTime.date, quantity, currentTime.time, 'Sell', currentTime.close]
        print('eday '+str(day)+' money: '+str(Money))
    
    ordersDf = pd.DataFrame(orders)
    ordersDf.columns = ['date','batches','time_op1','op1','value_op1','time_op2','op2','value_op2', 'profit', 'totalProfit', 'stop','Amount']
    return ordersDf

In [15]:
testPetr = runStrategy(pred_PETR4, initialDf_PETR4, 100000, 0.01, 0.02)
print('Accuracy: '+str(round(100*len(testPetr[testPetr['profit'] > 0].index)/len(testPetr.index), 2)))

testVale = runStrategy(pred_VALE3, initialDf_VALE3, 100000, 0.01, 0.02)
print('Accuracy: '+str(round(100*len(testVale[testVale['profit'] > 0].index)/len(testVale.index), 2)))

eday 0 money: 99454.00000000001
eday 1 money: 101635.99999999999
eday 2 money: 101195.99999999999
eday 3 money: 101679.99999999997
eday 4 money: 102269.99999999994
eday 5 money: 103017.99999999994
eday 6 money: 104979.99999999993
eday 7 money: 102867.99999999994
eday 8 money: 102559.99999999997
eday 9 money: 104382.99999999996
eday 10 money: 102282.99999999996
eday 11 money: 100728.99999999996
eday 12 money: 99972.99999999996
eday 13 money: 100812.99999999996
eday 14 money: 99331.99999999996
eday 15 money: 99326.99999999994
eday 16 money: 98162.99999999994
eday 17 money: 101780.99999999993
eday 18 money: 100948.99999999993
eday 19 money: 98589.99999999993
eday 20 money: 98460.99999999993
eday 21 money: 99350.99999999993
eday 22 money: 99538.99999999993
eday 23 money: 99495.99999999994
eday 24 money: 101495.99999999994
eday 25 money: 104837.99999999994
eday 26 money: 107315.99999999994
eday 27 money: 104535.99999999994
eday 28 money: 105937.99999999991
eday 29 money: 106219.99999999994


In [11]:
testPetr.tail(10)

,date,batches,time_op1,op1,value_op1,time_op2,op2,value_op2,profit,totalProfit,stop,Amount
61,2021.05.26,45,16:00:00,Sell,26.12,16:45:00,Buy,26.09,0.03,135.0,False,119401.0
62,2021.05.27,45,10:00:00,Sell,26.09,13:00:00,Buy,25.86,0.23,1035.0,False,120436.0
63,2021.05.28,45,10:30:00,Sell,26.57,11:30:00,Buy,26.84,-0.27,-1215.0,True,119221.0
64,2021.05.28,44,11:45:00,Sell,26.80,16:45:00,Buy,26.99,-0.19,-836.0,False,118385.0
65,2021.05.31,43,10:00:00,Sell,26.97,12:15:00,Buy,26.81,0.16,688.0,False,119073.0
66,2021.05.31,44,12:30:00,Buy,26.81,16:45:00,Sell,26.87,0.06,264.0,False,119337.0
67,2021.06.01,43,10:00:00,Buy,27.37,10:15:00,Sell,27.52,0.15,645.0,False,119982.0
68,2021.06.01,43,10:30:00,Buy,27.36,10:45:00,Sell,27.36,0.00,0.0,False,119982.0
69,2021.06.01,43,11:00:00,Sell,27.42,11:30:00,Buy,27.37,0.05,215.0,False,120197.0
70,2021.06.01,44,11:45:00,Buy,27.18,16:45:00,Sell,27.29,0.11,484.0,False,120681.0


In [12]:
testVale.tail(10)

,date,batches,time_op1,op1,value_op1,time_op2,op2,value_op2,profit,totalProfit,stop,Amount
49,2021.05.24,9,14:45:00,Sell,109.74,16:45:00,Buy,109.78,-0.04,-36.0,False,104058.0
50,2021.05.25,9,10:00:00,Sell,110.10,16:45:00,Buy,107.05,3.05,2745.0,False,106803.0
51,2021.05.26,10,10:00:00,Sell,106.65,10:30:00,Buy,107.83,-1.18,-1180.0,True,105623.0
52,2021.05.26,9,10:45:00,Sell,108.22,13:45:00,Buy,109.52,-1.30,-1170.0,True,104453.0
53,2021.05.26,9,14:00:00,Sell,109.52,16:45:00,Buy,110.20,-0.68,-612.0,False,103841.0
54,2021.05.27,9,10:00:00,Sell,111.35,16:45:00,Buy,110.99,0.36,324.0,False,104165.0
55,2021.05.28,9,10:00:00,Sell,111.15,14:30:00,Buy,112.70,-1.55,-1395.0,True,102770.0
56,2021.05.28,9,14:45:00,Sell,112.29,16:45:00,Buy,111.59,0.70,630.0,False,103400.0
57,2021.05.31,9,10:00:00,Sell,113.62,16:45:00,Buy,114.78,-1.16,-1044.0,True,102356.0
58,2021.06.01,8,10:00:00,Sell,117.54,16:45:00,Buy,113.20,4.34,3472.0,False,105828.0


In [13]:
df = pred_VALE3
dailyDf = initialDf_VALE3
size = len(df.index)

Money = 100000
stopLoss = 0.01
LastTrade = ['None',10,100]

orders = []

for day in range(0,size):
    currentDay =  df.iloc[day].date

    lowPred = df.iloc[day].low_pred*1.01
    highPred = df.iloc[day].high_pred*0.99
    
    dayDf = getPeriod2(dailyDf, currentDay, currentDay)
    daySize = len(dayDf.index)
    
    op = 'none'
    quantity = 0
    trade = []
    
    print('Begining day '+str(day)+' money: '+str(Money))
    for i in range(0,daySize):
        currentTime = dayDf.iloc[i]
        
        if((op == 'buy') and (currentTime.close < opPrice*(1-stopLoss) or currentTime.close >= highPred or i == daySize-1)):
            isStop = False
            if(currentTime.close < opPrice*(1-stopLoss)): isStop = True
            aux = sell(currentTime.close, quantity)
            Money = Money + aux 
            trade = trade + [currentTime.time, 'Sell', currentTime.close, currentTime.close-opPrice,(currentTime.close-opPrice)*100*quantity, isStop, Money]
            orders.append(trade)
  
            quantity = 0
            op = 'none'
        
        elif((op == 'sell') and (currentTime.close > opPrice*(1+stopLoss) or currentTime.close <= lowPred or i == daySize-1)):
            isStop = False
            if(currentTime.close > opPrice*(1+stopLoss)): isStop = True

            aux = sell(currentTime.close, quantity)
            Money = Money + (opPrice*100*quantity) + ((opPrice*100*quantity) - aux) 
            trade = trade + [currentTime.time, 'Buy', currentTime.close, opPrice-currentTime.close,(opPrice-currentTime.close)*100*quantity, isStop, Money]
            orders.append(trade)
            
            quantity = 0
            op = 'none'
        
        elif(op == 'none' and i < daySize-1 ):
            if(currentTime.close <= lowPred):
                Money, quantity = buy(currentTime.close, Money)
                opPrice = currentTime.close
                op = 'buy'
                
                trade = [currentTime.date, quantity, currentTime.time, 'Buy', currentTime.close]
                
            elif(currentTime.close >=  highPred):
                Money, quantity = buy(currentTime.close, Money)
                opPrice = currentTime.close
                op = 'sell'
                
                trade = [currentTime.date, quantity, currentTime.time, 'Sell', currentTime.close]

test2 = pd.DataFrame(orders)
test2.columns = ['date','batches','time_op1','op1','value_op1','time_op2','op2','value_op2', 'profit', 'totalProfit', 'stop','Amount']
test2.tail(60)

test2 = pd.DataFrame(orders)
test2.columns = ['date','batches','time_op1','op1','value_op1','time_op2','op2','value_op2', 'profit', 'totalProfit', 'stop','Amount']

print('Accuracy: '+str(round(100*len(test2[test2['profit'] > 0].index)/len(test2.index), 2)))
test2.head(10)

Begining day 0 money: 100000
Begining day 1 money: 101548.0
Begining day 2 money: 100225.0
Begining day 3 money: 100864.0
Begining day 4 money: 101611.0
Begining day 5 money: 101053.0
Begining day 6 money: 101494.0
Begining day 7 money: 99325.0
Begining day 8 money: 100459.0
Begining day 9 money: 100261.0
Begining day 10 money: 101944.0
Begining day 11 money: 103375.0
Begining day 12 money: 103888.0
Begining day 13 money: 103987.0
Begining day 14 money: 104860.00000000001
Begining day 15 money: 103411.00000000001
Begining day 16 money: 101494.00000000001
Begining day 17 money: 102214.00000000001
Begining day 18 money: 102628.00000000001
Begining day 19 money: 103726.00000000001
Begining day 20 money: 103330.00000000001
Begining day 21 money: 104734.00000000001
Begining day 22 money: 102601.00000000001
Begining day 23 money: 103345.00000000001
Begining day 24 money: 107089.00000000001
Begining day 25 money: 103568.00000000004
Begining day 26 money: 106560.00000000004
Begining day 27 mon

,date,batches,time_op1,op1,value_op1,time_op2,op2,value_op2,profit,totalProfit,stop,Amount
0,2021.04.06,9,10:00:00,Sell,103.77,16:45:00,Buy,102.05,1.72,1548.0,False,101548.0
1,2021.04.07,9,10:00:00,Sell,102.76,13:30:00,Buy,103.95,-1.19,-1071.0,True,100477.0
2,2021.04.07,9,13:45:00,Sell,104.28,16:45:00,Buy,104.56,-0.28,-252.0,False,100225.0
3,2021.04.08,9,10:00:00,Sell,105.21,16:45:00,Buy,104.50,0.71,639.0,False,100864.0
4,2021.04.09,9,10:00:00,Sell,103.83,16:45:00,Buy,103.00,0.83,747.0,False,101611.0
5,2021.04.12,9,10:00:00,Sell,102.78,16:45:00,Buy,103.40,-0.62,-558.0,False,101053.0
6,2021.04.13,9,10:00:00,Sell,104.07,16:45:00,Buy,103.58,0.49,441.0,False,101494.0
7,2021.04.14,9,10:00:00,Sell,104.18,10:45:00,Buy,105.61,-1.43,-1287.0,True,100207.0
8,2021.04.14,9,11:00:00,Sell,106.00,11:30:00,Buy,107.44,-1.44,-1296.0,True,98911.0
9,2021.04.14,9,11:45:00,Sell,107.46,16:45:00,Buy,107.00,0.46,414.0,False,99325.0
